In [1]:
import os
import re
import csv
import pandas as pd
import json
import numpy as np
import heapq
from matplotlib import colors
import matplotlib.pyplot as plt
import torch
import networkx as nx
from networkx.readwrite import json_graph
from graph_to_prompt import graph_to_prompt
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm

In [2]:
def get_nums_list(s):
    nums_str = re.findall(r'\d+', s)  
    nums = list(map(int, nums_str))
    return  nums

In [3]:
class Request_llm:
    def __init__(self, model_name):
        if model_name == 'llama':
            self.max_new_tokens = 200
            self.tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
            self.model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", device_map="auto",load_in_8bit=True)
            #self.model.save_model("models/meta-llama/Llama-3.2-3B-Instruct")

        if model_name == 'deepseek':
            self.max_new_tokens = 50
            self.tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")
            self.model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B", device_map="auto",load_in_8bit=True)

        if model_name == 'qwen':
            self.tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")
            self.model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B", device_map="auto",load_in_8bit=True) 

        self.model.config.pad_token_id = self.model.config.eos_token_id

    def get_waypoints(self, j, start, goal, n_points):
        prompt = graph_to_prompt(j, start, goal, n_points)
        if.
        inputs = self.tokenizer(
                    prompt,
                    return_tensors="pt",
                    return_attention_mask=True,
                    truncation=True     # 입력도 길면 자르기
                ).to(self.model.device)
        
        input_ids = inputs["input_ids"]
        inputs["input_ids"] = inputs["input_ids"].to(self.model.device)
        inputs["attention_mask"] = inputs["attention_mask"].to(self.model.device)

        outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,  # ← 필수
                num_beams=1,
                use_cache=True,
                do_sample=False,
            )
        generated_token = outputs[:, input_ids.shape[-1] :]
        result = self.tokenizer.batch_decode(generated_token, skip_special_tokens=True)[0]
        waypoints = get_nums_list(result)
        return waypoints


SyntaxError: invalid syntax (1605473239.py, line 22)

In [ ]:
def astar(G, start, goal, heuristic, obstacles = None):
    count = 0
    storage = 0
    checking_edges = set()

    if obstacles is None:
        obstacles = set()
    else:
        obstacles = set(obstacles)

    # g-score 와 f-score 초기화
    g_score = {n: float('inf') for n in G.nodes}
    g_score[start] = 0

    f_score = {n: float('inf') for n in G.nodes}
    f_score[start] = heuristic[start]

    # 우선순위 큐: (f_score, g_score, node)
    open_set = []
    heapq.heappush(open_set, (f_score[start], 0, start))
    visited = set()
    came_from = {}

    num_iter = 0

    while open_set:
        num_iter += 1
        storage += len(open_set) + len(visited) + len(came_from)
        f_cur, g_cur, current = heapq.heappop(open_set)

        # 목표 도달
        if current == goal:
            # 경로 재구성
            path = [current]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            return path[::-1], g_cur, count, storage, list(checking_edges)
        visited.add(current)
        # 이웃 탐색
        for nbr in G.neighbors(current):
            count = count + 1
            if nbr in visited or nbr in obstacles:
                continue
            
            tentative_g = g_score[current] + G[current][nbr]['weight']
            checking_edges.add((current, nbr))
            if tentative_g < g_score[nbr]:
                came_from[nbr] = current
                g_score[nbr] = tentative_g
                f_score[nbr] = tentative_g + heuristic[nbr]
                heapq.heappush(open_set, (f_score[nbr], tentative_g, nbr))

    # 경로가 없을 때
    return None, None, count, storage, list(checking_edges)

In [ ]:
def llm_astar(G, start, goal, heuristic, llm, obstacles=None):
    nodes = list(G.nodes)
    llm = list(set(llm))
    T = []
    for ts in llm:
        if ts in nodes:
            T.append(ts)
    if obstacles is None:
        obstacles = set()
    else:
        obstacles = set(obstacles)
    print(llm)
    # T.reverse()
    if not T or T[0] != start:
        T.insert(0, start)
    if T[-1] != goal:
        T.append(goal)
    T = [t for t in T if t not in obstacles]
    t_idx = 1 if len(T) > 1 else 0
    t = T[t_idx]
    dist_to_t = nx.single_source_dijkstra_path_length(G, t, weight='weight')
    count = 0
    storage = 0
    checking_edges = set()
    came_from = {}
    visited = set()

    g_score = {n: float('inf') for n in G.nodes}
    g_score[start] = 0

    f_score = {n: float('inf') for n in G.nodes}
    f_score[start] = g_score[start] + heuristic[start] + dist_to_t.get(start, float('inf'))

    open_set = []
    heapq.heappush(open_set, (f_score[start], g_score[start], start))

    while open_set:
        storage += len(open_set) + len(visited) + len(came_from)
        f_cur, g_cur, current = heapq.heappop(open_set)
        if current == goal:
            path = [current]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            return path[::-1], g_cur, count, storage, list(checking_edges), T
        visited.add(current)
        if current == t and t_idx < len(T) - 1:
            t_idx += 1
            t = T[t_idx]
            dist_to_t = nx.single_source_dijkstra_path_length(G, t, weight='weight')

            new_open = []
            for _, g_old, node in open_set:
                new_f = g_score[node] + heuristic[node] + dist_to_t.get(node, float('inf'))
                new_open.append((new_f, g_old, node))
            open_set = new_open
            heapq.heapify(open_set)


        for nbr in G.successors(current):
            if nbr in visited or nbr in obstacles:
                continue
            count += 1
            w = G[current][nbr].get('weight', 1)
            tentative_g = g_score[current] + w
            checking_edges.add((current, nbr))

            if tentative_g < g_score[nbr]:
                came_from[nbr] = current
                g_score[nbr] = tentative_g
                f = tentative_g + heuristic[nbr] + dist_to_t.get(nbr, float('inf'))
                f_score[nbr] = f
                heapq.heappush(open_set, (f, tentative_g, nbr))


    return None, None, count, storage, list(checking_edges), T


In [8]:
with open("graphs/sejong_bus.json", "r") as json_file:
    j = json.load(json_file)

G = json_graph.node_link_graph(j)
nodes = G.nodes
edges = G.edges
stations = list(G.nodes())

/opt/conda/envs/LLM/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [5]:
# table(random) A*
experiments_samples_A_star = []

while True:
    start, goal = np.random.choice(nodes, 2)
    start, goal = start.item(), goal.item()

    heuristic_table = {station: (((G.nodes[goal]['x'] - G.nodes[station]['x']) ** 2 + (G.nodes[goal]['y'] - G.nodes[station]['y']) ** 2) ** 0.5) for station in nodes}

    path, cost, count, storage, checking_edges = astar(G, start, goal, heuristic_table)
    if path is not None:
        experiments_samples_A_star.append({'point' : (start, goal), 'path':path, 'cost':cost, 'storage':storage, 'count':count, 'checking_edges' : len(checking_edges)})
    if len(experiments_samples_A_star)>150:
        break

NameError: name 'nodes' is not defined

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B", device_map='auto', load_in_8bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
start, goal = 6800326, 6800463

n_points = 7
prompt = graph_to_prompt(j, start, goal, n_points)

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True).to(model.device)
        
input_ids = inputs["input_ids"]
inputs["input_ids"] = inputs["input_ids"].to(model.device)
inputs["attention_mask"] = inputs["attention_mask"].to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    )
generated_token = outputs[:, input_ids.shape[-1] :]
result = tokenizer.batch_decode(generated_token, skip_special_tokens=True)[0]
print(result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


[6800376, 6800377, 6800378, 6800379, 6800380, 6800381, 6800382, 6800383, 6800384, 6800385, 6800386, 


In [ ]:

model.config.max_position_embeddings
llm_model = Request_llm('deepseek')

In [ ]:
n_points = 7
prompt = graph_to_prompt(j, start, goal, n_points)
llm_model.tokenizer(prompt)

In [ ]:
# table(random) LLM-A*
num_request = 7
llm_model.model.eval()
ctx = torch.inference_mode  # 또는 torch.no_grad
experiments_samples_LLM_A_star = []

with ctx():
    for i, sample in (enumerate(tqdm(experiments_samples_A_star[:10]))):
        start, goal = sample['point']

        heuristic_table = {station: (((G.nodes[goal]['x'] - G.nodes[station]['x']) ** 2 + (G.nodes[goal]['y'] - G.nodes[station]['y']) ** 2) ** 0.5) for station in nodes}

        waypoints = llm_model.get_waypoints(j, start, goal, num_request)
        
        path, cost, count, storage, checking_edges, waypoints_using_llm = llm_astar(G, start, goal, heuristic_table, waypoints)
        if path is not None:
            experiments_samples_LLM_A_star.append(
                {'point' : (start, goal), 
                'path':path, 
                'cost':cost, 
                'count':count, 
                'storage':storage,
                'checking_edges' : len(checking_edges),
                'waypoints_of_llm' : waypoints,
                'waypoints_using_llm' : waypoints_using_llm,
                'success' : (len(waypoints) <= num_request + 1)
                })
            cost_ratio = 1 - cost/sample['cost']
            count_ratio = 1 - count/sample['count']
            storage_ratio = 1 - storage/sample['storage']
            checking_edges_ratio = 1 - len(checking_edges)/sample['checking_edges']
            print("Sample ", i)
            print("Count ratio       :", count_ratio)
            print("Storage ratio    ",  storage_ratio)
            print("Cost ratio        :", cost_ratio)
            print("# of Checking edges ratio:", checking_edges_ratio)

In [ ]:
data = experiments_samples_LLM_A_star

fields = ['point', 'path', 'cost', 'count', 'checking_edges', 'waypoints_of_llm', 'waypoints_using_llm', 'success']

with open('Experiments_result/LLM_A_star_fewshot.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

In [ ]:
data = pd.read_csv('Experiments_result/A_star.csv')
data.loc[0].

In [ ]:
data = []

In [ ]:
import ast
ast.literal_eval(data[0]['point'])

In [ ]:
experiments_samples_A_star[0]

In [ ]:
with open('Experiments_result/A_star.json', 'w') as jsonfile:
	json.dump(experiments_samples_A_star, jsonfile, indent=4)

In [ ]:
with open('Experiments_result/A_star.json', 'r') as jsonfile:
    data = json.load(jsonfile)

print(data)